In [1]:
import laser_lib
import numpy as np
import math
import time
#import cv2
#from skimage.color import rgb2lab, lab2rgb

In [2]:
queue = laser_lib.DacQueue()

Found  1 Helios DACs


In [3]:
def signer(f, T, phase=0.0):
    '''
    Produces spaced out points from 0 to 1 whose distance between 
    consecutive points is sinusoidal 
    Input:  f - frequency 
            T - number of samples
            phase - phase shift (0, 1)
    Returns: x - positions on the line
             dx - the derivative of the function of x
    '''
    # Convert to radians
    f = f*2*np.pi
    # Generate the baseline 
    t = np.linspace(0.0+phase, 1.0+phase, T)
    # Find points 
    x = (f*t - np.cos(f*t) + 1)/f
    # Calculate derivative
    dx = (np.sin(f*t) + 1)/2
    
    return x, dx

def triangle(theta, alpha, sincos='sin'):
    '''
    Returns a wave between a sine wave and triangle wave depending on alpha
    alpha = 0 - sine wave
    alpha = 1 - triangle wave
    '''
    if(sincos == 'sin'):
        ps = 0
    elif(sincos == 'cos'):
        ps = np.pi/2
    return alpha*2*np.arcsin(np.sin(theta + ps))/np.pi + (1-alpha)*np.sin(theta + ps)
    
def ellipser(scale_x=1, scale_y=1, f_x=1, f_y=1, f_s=1000, phase_s=0, sintriang=0, colors=[(1,1,1)], T=500):
    '''
    Generates a pattern that is a function of ellipses 
    with parameters that can be interpolated between
    '''
    theta_norm, theta_dx = signer(f=f_s, phase=phase_s, T=T)
    theta = theta_norm*2*np.pi
    arr_pos = np.zeros((T, 2))
    arr_pos[:,0] = scale_x*triangle(f_x*theta, sintriang, 'sin')
    arr_pos[:,1] = scale_y*triangle(f_y*theta, sintriang, 'cos')
    
    # Color the pattern
    arr_col = np.ones((T, 3), np.float32)
    if(len(colors) == 1):
        # All one color
        arr_col[:, :] = colors[0][np.newaxis, :]
    if(len(colors) == 2):
        # Color based on speed of laser
        arr_col[theta_dx > 0.1, :] = colors[0]
        arr_col[theta_dx <= 0.1, :] = colors[1]
    
    return arr_pos, arr_col


In [6]:
# CIELAB COLOR SPACE
ab_min = -127
ab_max = 128
l_min, l_max  = (0, 130)
num_points = ab_max - ab_min + 1

l = np.linspace(l_min, l_max, num_points)
a = np.linspace(ab_min, ab_max, num_points)
b = np.linspace(ab_min, ab_max, num_points)
l, a, b = np.meshgrid(l, a, b, indexing='ij')
lab = np.stack([l, a, b], axis=-1)

# Convert LAB coords to RGB
rgb = lab2rgb(lab)

for lum in np.concatenate([np.arange(num_points), np.arange(num_points)[::-1]]):
    frame = rgb[lum, :, :, :]
    print('Lum', lum, lab[lum, 0])
    cv2.imshow('frame', cv2.resize(frame, None, fx=2, fy=2))
    cv2.waitKey(10)


NameError: name 'lab2rgb' is not defined

In [ ]:
# Sampling the CIELAB Space
AB_choices = np.arange(start, stop+1).astype(np.float32)
while(True):
    # Choose a Lum
    lum_c = int(100*np.random.rand(1))
    # Choose an A
    a_c = np.random.choice(AB_choices)
    # Choose a B 
    b_c = np.random.choice(AB_choices)

    try:
        point_rgb = lab2rgb([[lum_c, a_c, b_c]])
        point_rgb_opp = lab2rgb((lum_c, -1*a_c, -1*b_c))
    except:
        # Skip this invalid combo
        continue
    print((lum_c, a_c, b_c), point_rgb)
    

    frame = np.zeros((400, 200, 3))
    frame[0:200, :, :] = point_rgb
    frame[200:400, :, :] = point_rgb_opp
    cv2.imshow('point', frame)
    cv2.waitKey(1000)


In [4]:
# Make spiral of CIELab
T = 1000
turns = 7
theta = np.linspace(0, 2*np.pi*turns, T)
scale = np.linspace(0, 1, T)

arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = scale*np.cos(theta)
arr_pos[:, 1] = scale*np.sin(theta)

arr_col_lab = np.zeros((T, 3))
arr_col_lab[:, 0] = (l_max-l_min)*np.sqrt(arr_pos[:, 0]**2 + arr_pos[:, 1]**2) + l_min
w = (l_max-l_min)*scale + l_min
arr_col_lab[:, 1] = (ab_max-ab_min)*(arr_pos[:, 0]+1)/2 + ab_min
arr_col_lab[:, 2] = (ab_max-ab_min)*(arr_pos[:, 1]+1)/2 + ab_min

arr_col_rgb = lab2rgb(arr_col_lab)

queue.dac_rate = 40000
for step in range(1000):
    queue.submit(arr_pos, arr_col_rgb)
    #queue.submit(arr_pos[::-1], arr_col_rgb[::-1])

NameError: name 'l_max' is not defined

In [6]:
# Circle
queue.dac_rate = 30000
T = 500
arr_pos = np.zeros((T, 2))
arr_col = np.zeros((T, 3))
arr_col[:, :] = 1
theta = np.linspace(0, 2*np.pi, T)
arr_pos[:, 0] = 0.5*np.cos(theta)
arr_pos[:, 1] = 0.5*np.sin(theta)

#arr_col = arr_col*np.expand_dims((theta > np.pi/2), 1)

while(True):
    queue.submit(arr_pos, arr_col, loop=False)
queue.submit(arr_pos, np.zeros((T,3)), loop=True)

KeyboardInterrupt: 

In [4]:
# Circle
queue.dac_rate = 30000
T = 300
print('Circle Frequency: {0}Hz'.format(queue.dac_rate / T))
arr_pos = np.zeros((T, 2))
arr_col = np.zeros((T, 3))
#arr_col[:, :] = 1

theta = np.linspace(0, 2*np.pi, T)
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

eps = np.pi/40
mask1 = np.logical_and((theta > (np.pi/2 - eps/2)), (theta < (np.pi/2 + eps/2)))
mask2 = np.logical_and((theta > (3*np.pi/2 - eps/2)), (theta < (3*np.pi/2 + eps/2)))
#arr_col = arr_col*np.expand_dims(mask1 + mask2, 1)
arr_col[:, 2] = mask1 
arr_col[:, 1] = mask2
print(mask2)

while(True):
    queue.submit(arr_pos, arr_col)

Circle Frequency: 100.0Hz
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False 

KeyboardInterrupt: 

In [ ]:
# Interpolated ellipses
T = 1000
arr_col = np.ones((T, 3))
theta = np.linspace(0, 2*np.pi, T)

queue.dac_rate = 45000

min_vals = np.array([0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
max_vals = np.array([1.0, 0.4, 2.0, 3.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
pat_point_old = (max_vals-min_vals)*np.random.rand(len(max_vals)) + min_vals


T_trans = 150

while(True):
    pat_point = (max_vals-min_vals)*np.random.rand(len(max_vals)) + min_vals
    ell_interp = np.linspace(pat_point_old, pat_point, T_trans)
    for (scale_x, scale_y, freq_x, freq_y, f_s, phase, sintriang, r1, g1, b1, r2, g2, b2) in ell_interp:
        # Transform the distribution of f_s to be less uniform
        f_s = 40*np.power(f_s, 6)
        #TODO - transform sintriang to have a bathtub curve
        
        # Get the array
        arr_pos, arr_col = ellipser(scale_x=scale_x, 
                                    scale_y=scale_y, 
                                    f_x=freq_x, 
                                    f_y=freq_y, 
                                    f_s=f_s, 
                                    phase_s=phase,
                                    sintriang=sintriang, 
                                    colors=[(r1,g1,b1), (r2,g2,b2)],
                                    T=T)
        
        queue.submit(arr_pos, np.roll(arr_col, shift=9, axis=0))
        
    pat_point_old = pat_point

In [11]:
# Cycling back and forth between ellipses
T = 1000
queue.dac_rate = 45000

min_vals = np.array([0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
max_vals = np.array([1.0, 0.4, 2.0, 3.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])


T_trans = 50

repeat_cycle_count = 4

while(True):
    # Pick two points to cycle around
    pat_point_a = (max_vals-min_vals)*np.random.rand(len(max_vals)) + min_vals
    pat_point_b = (max_vals-min_vals)*np.random.rand(len(max_vals)) + min_vals
    # Go there and come back
    ell_interp = np.linspace(pat_point_a, pat_point_b, T_trans//2)
    ell_interp = np.concatenate([ell_interp, ell_interp[::-1, :]])
    for r in range(repeat_cycle_count):
        for (scale_x, scale_y, freq_x, freq_y, f_s, phase, sintriang, r1, g1, b1, r2, g2, b2) in ell_interp:
            # Transform the distribution of f_s to be less uniform
            f_s = 40*np.power(f_s, 6)
            #TODO - transform sintriang to have a bathtub curve

            # Get the array
            arr_pos, arr_col = ellipser(scale_x=scale_x, 
                                        scale_y=scale_y, 
                                        f_x=freq_x, 
                                        f_y=freq_y, 
                                        f_s=f_s, 
                                        phase_s=phase,
                                        sintriang=sintriang, 
                                        colors=[(r1,g1,b1), (r2,g2,b2)],
                                        T=T)

            queue.submit(arr_pos, np.roll(arr_col, shift=9, axis=0))
        


KeyboardInterrupt: 

In [12]:
T = 750
arr_col = np.ones((T, 3))
theta = np.linspace(0, 2*np.pi, T)



(r1,g1,b1) = (0, 0.1, 0.1)
(r2,g2,b2) = (1.0, 0.4, 0.4)
for shift in range(-10, 10):
    print(shift)
    for i in range(100):
        arr_pos, arr_col = ellipser(scale_x=1.0, 
                                    scale_y=1.0, 
                                    f_x=1.0, 
                                    f_y=1.0, 
                                    f_s=10, 
                                    phase_s=0.0,
                                    sintriang=0.0, 
                                    colors=[(r1,g1,b1), (r2,g2,b2)],
                                    T=T)

        queue.submit(arr_pos, np.roll(arr_col, shift, axis=0))
        


-10
-9
-8
-7
-6


KeyboardInterrupt: 

In [13]:
# Align mirrors and laser 
T = 1000
turns = 7
theta = np.linspace(0, 2*np.pi*turns, T)

arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.linspace(-1.0, 1.0, T)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.zeros((T, 3))
#arr_col[:, 0] = np.abs(arr_pos[:, 1]) > 0.99
arr_col[:, 0] = np.abs(arr_pos[:, 1]) > 0.999
arr_col[:, 1] = np.abs(arr_pos[:, 1]) > 0.999
arr_col[:, 2] = np.abs(arr_pos[:, 1]) > 0.999

# Corresponds to Red, Green, Blue @ dac_rate 55k
shifts = [11, 10, 9]

# Correct color array by shifting
#for col_idx  in range(3):
#    arr_col[:, col_idx] = np.roll(arr_col[:, col_idx], shifts[col_idx], axis=0)
    
#for col_idx, shift in enumerate(shifts):
#    print(col_idx, shift)
#    arr_col[:, col_idx] = np.roll(arr_col[:, col_idx], shift, axis=0)

for shift in range(8, 20):
    print(shift)
    for i in range(100):
        queue.submit(arr_pos, arr_col)

8
9


KeyboardInterrupt: 

In [ ]:
# Circle
queue.dac_rate = 30000
T = 500
arr_pos = np.zeros((T, 2))
arr_col = np.zeros((T, 3))
arr_col[:, :] = 1
theta = np.linspace(0, 2*np.pi, T)
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)


while(True):
    queue.submit(arr_pos, arr_col/6)